## Bibliotecas Utilizadas

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D, AveragePooling2D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import plot_model
from keras import backend as K
from keras.preprocessing.image import img_to_array, load_img
from datetime import datetime, time

In [4]:
print ("Início Execução : ")
print (datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Início Execução : 
2021-07-21 23:34:11


# Funções

In [30]:
def get_dados(path, df):
    X = []
    Y = []
    for index, row in df.iterrows():
            arquivo = '{0}/{1}'.format(path, row["X_ray_image_name"])
            Xray = img_to_array(load_img(arquivo, target_size=(28, 28)))
            X.append(Xray)
            Y.append(row["Label"])
    return (X, Y)

In [6]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training Accuracy')
    plt.plot(x, val_acc, 'r', label='Validation Accuracy')
    plt.title('Training and validation Accuracy')
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [7]:
def CNN_PaperOriginal(num_frutas, size, camadas):
    # Cria o modelo
    model = Sequential()
    
    #Convolução 2D com função de ativação Rectified Linear Units 32 kernels/Pesos (filtros) 
    model.add(Conv2D(4, (5, 5), input_shape=(size, size, camadas), activation='relu'))#
    print( model.output_shape)
    #Camada de Pooling 	    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    print(model.output_shape)
    
    model.add(Dropout(0.2))

    model.add(Conv2D(16, (5, 5), activation='relu'))#
    print(model.output_shape)
    #Camada de Pooling 	    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    print(model.output_shape)
    
    model.add(Dropout(0.2))
    
    model.add(Conv2D(32, (5, 5), activation='relu'))#
    print(model.output_shape)
    #Camada de Pooling 	    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    print(model.output_shape)
    
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64, (5, 5), activation='relu'))#
    print(model.output_shape)
    #Camada de Pooling 	    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    print(model.output_shape)
    
    model.add(Dropout(0.2))
    
    #Converte o conjunto de imagens e um vetor unidimensional para a entrada da rede neural totalmente conectada
    model.add(Flatten())
    print(model.output_shape)
    model.add(Dense(1024, activation='tanh'))
    print(model.output_shape)
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='tanh'))
    print(model.output_shape)
    model.add(Dropout(0.2))
    model.add(Dense(60, activation='tanh'))
    print(model.output_shape)
    model.add(Dropout(0.2))
    #Função de Ativação Softmax
    model.add(Dense(num_frutas, activation='softmax'))
    print(model.output_shape)
    return model

In [8]:
def CNN_LeNet5(num_frutas,size, camadas):
    # Cria o modelo
    model = Sequential()
    #Convolução 2D com função de ativação Rectified Linear Units 32 kernels/Pesos (filtros) 
    model.add(Conv2D(6, (5, 5), input_shape=(size, size, camadas), activation='relu'))#
    print( model.output_shape)
    #Camada de Pooling 	    
    model.add(AveragePooling2D(pool_size=(2, 2)))
    print( model.output_shape)
    
    model.add(Dropout(0.2))

    model.add(Conv2D(16, (5, 5), activation='relu'))#
    print( model.output_shape)
    #Camada de Pooling 	    
    model.add(AveragePooling2D(pool_size=(2, 2)))
    print( model.output_shape)
    
    model.add(Dropout(0.2))

    #Converte o conjunto de imagens e um vetor unidimensional para a entrada da rede neural totalmente conectada
    model.add(Flatten())
    print(model.output_shape)
    model.add(Dense(240, activation='tanh'))
    print(model.output_shape)
    model.add(Dropout(0.2))    
    model.add(Dense(120, activation='tanh'))
    print(model.output_shape)
    model.add(Dropout(0.2))    
    #Função de Ativação Softmax
    model.add(Dense(num_frutas, activation='softmax'))
    print(model.output_shape)
    return model

In [9]:
def CNN_AlexNet(num_frutas,size,camadas):
    # Cria o modelo
    model = Sequential()
    #Convolução 2D com função de ativação Rectified Linear Units 32 kernels/Pesos (filtros) 
    model.add(Conv2D(96, (11, 11), input_shape=(size, size, camadas), activation='relu'))#
    print(model.output_shape)
    #Camada de Pooling 	    
    model.add(MaxPooling2D(pool_size=(3, 3)))
    print(model.output_shape)
    
    model.add(Dropout(0.2))

    model.add(Conv2D(256, (5, 5), activation='relu'))#
    print(model.output_shape)
    #Camada de Pooling 	    
    model.add(MaxPooling2D(pool_size=(3, 3)))
    print(model.output_shape)
    
    model.add(Dropout(0.2))

    model.add(Conv2D(384, (3, 3), activation='relu'))#
    print(model.output_shape)
    model.add(Dropout(0.2))

    model.add(Conv2D(384, (3, 3), activation='relu'))#
    print(model.output_shape)
    model.add(Dropout(0.2))

    model.add(Conv2D(256, (3, 3), activation='relu'))#
    print(model.output_shape)
    model.add(Dropout(0.2))

    #Converte o conjunto de imagens e um vetor unidimensional para a entrada da rede neural totalmente conectada
    model.add(Flatten())
    print(model.output_shape)
    model.add(Dense(512, activation='relu'))
    print(model.output_shape)
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    print(model.output_shape)
    model.add(Dropout(0.2))
    #Função de Ativação Softmax
    model.add(Dense(num_frutas, activation='softmax'))
    print(model.output_shape)
    return model

# Rodando Algoritmos

In [10]:
path_treino = './X-Rays/train' 
path_teste = './X-Rays/test'

In [11]:
XRay = pd.read_csv('./X-Rays/Chest_xray_Corona_Metadata.csv')
XRay.head()

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN
1,1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN
2,2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN
3,3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN
4,4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN


In [12]:
XRayTreino = XRay[XRay["Dataset_type"]=='TRAIN']
XRayTest = XRay[XRay["Dataset_type"]=='TEST']

In [13]:
XRay.size

35460

In [14]:
XRayTreino.size

31716

In [15]:
XRayTest.size

3744

In [16]:
XRayTreino.size + XRayTest.size

35460

In [40]:
X_Ray, Result = get_dados(path_treino, XRayTreino)
X_train = np.asarray(X_Ray).astype(np.float32)
Y_train = np.asarray(Result)
X_train = X_train.astype('float32')/255
num_cateogorias = len(list(set(Y_train)))
print(num_cateogorias)
Y_train = to_categorical(Y_train, num_cateogorias)

#X_train, x_val, Y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

2


ValueError: invalid literal for int() with base 10: 'Normal'

In [37]:
X_Train[0]

array([[[  0.,   0.,   0.],
        [ 23.,  23.,  23.],
        [ 69.,  69.,  69.],
        ...,
        [ 70.,  70.,  70.],
        [ 35.,  35.,  35.],
        [ 12.,  12.,  12.]],

       [[  0.,   0.,   0.],
        [ 23.,  23.,  23.],
        [ 80.,  80.,  80.],
        ...,
        [ 77.,  77.,  77.],
        [ 19.,  19.,  19.],
        [  1.,   1.,   1.]],

       [[  0.,   0.,   0.],
        [ 17.,  17.,  17.],
        [ 37.,  37.,  37.],
        ...,
        [ 92.,  92.,  92.],
        [ 30.,  30.,  30.],
        [  0.,   0.,   0.]],

       ...,

       [[  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        ...,
        [ 95.,  95.,  95.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.]],

       [[  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        ...,
        [102., 102., 102.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.]],

       [[  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,  

In [13]:
X_train = np.asarray(X_frutas).astype(np.float32)
Y_train = np.asarray(Y_frutas)
X_train = X_train.astype('float32')/255
num_frutas = len(list(set(Y_train)))
print(num_frutas)
Y_train = to_categorical(Y_train, num_frutas)

X_train, x_val, Y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

0 - Apple Braeburn
1 - Apple Crimson Snow
2 - Apple Golden 1
3 - Apple Golden 2
4 - Apple Golden 3
5 - Apple Granny Smith
6 - Apple Pink Lady
7 - Apple Red 1
8 - Apple Red 2
9 - Apple Red 3
10 - Apple Red Delicious
11 - Apple Red Yellow 1
12 - Apple Red Yellow 2
13 - Apricot
14 - Avocado
15 - Avocado ripe
16 - Banana
17 - Banana Lady Finger
18 - Banana Red
19 - Beetroot
20 - Blueberry
21 - Cactus fruit
22 - Cantaloupe 1
23 - Cantaloupe 2
24 - Carambula
25 - Cauliflower
26 - Cherry 1
27 - Cherry 2
28 - Cherry Rainier
29 - Cherry Wax Black
30 - Cherry Wax Red
31 - Cherry Wax Yellow
32 - Chestnut
33 - Clementine
34 - Cocos
35 - Dates
36 - Eggplant
37 - Ginger Root
38 - Granadilla
39 - Grape Blue
40 - Grape Pink
41 - Grape White
42 - Grape White 2
43 - Grape White 3
44 - Grape White 4
45 - Grapefruit Pink
46 - Grapefruit White
47 - Guava
48 - Hazelnut
49 - Huckleberry
50 - Kaki
51 - Kiwi
52 - Kohlrabi
53 - Kumquats
54 - Lemon
55 - Lemon Meyer
56 - Limes
57 - Lychee
58 - Mandarine
59 - Mang

MemoryError: Unable to allocate 6.76 GiB for an array with shape (60498, 100, 100, 3) and data type float32

In [ ]:
model = CNN_PaperOriginal(num_frutas,100,3)
#plot_model(model, show_shapes=True, show_layer_names=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
checkpoint = ModelCheckpoint(filepath='./modelo_cnn_8_1.hdf5', verbose=0,  save_best_only=True, monitor='val_accuracy')
history = model.fit(X_train, Y_train, validation_data = (x_val, y_val), epochs=50, batch_size=60, callbacks=[checkpoint])
plot_history(history)
scores = model.evaluate(X_train, Y_train, verbose=0)
print("Erro da CNN: %.2f%%" % (100-scores[1]*100))
print("Acurácia da CNN: %.2f%%" % (scores[1]*100))
X_train = None
Y_train = None
x_val = None
y_val = None

# Gray Scale

In [ ]:
X_frutas, Y_frutas = get_dados_grayscale(path_treino, 28)
X_train_grayscale = np.asarray(X_frutas).astype(np.float32)
Y_train_grayscale = np.asarray(Y_frutas)
X_train_grayscale = X_train_grayscale.astype('float32')/255
num_frutas = len(list(set(Y_train_grayscale)))
print(num_frutas)
Y_train_grayscale = to_categorical(Y_train_grayscale, num_frutas)

X_train_grayscale, x_val_grayscale, Y_train_grayscale, y_val_grayscale = train_test_split(X_train_grayscale, Y_train_grayscale, test_size=0.2, random_state=42)

In [ ]:
model = CNN_LeNet5(num_frutas,28,1)
#plot_model(model, show_shapes=True, show_layer_names=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
checkpoint = ModelCheckpoint(filepath='./modelo_cnn_LeNet_8_1.hdf5', verbose=0,  save_best_only=True, monitor='val_accuracy')
history = model.fit(X_train_grayscale, Y_train_grayscale, validation_data = (x_val_grayscale, y_val_grayscale), epochs=50, batch_size=60, callbacks=[checkpoint])
plot_history(history)
scores = model.evaluate(X_train_grayscale, Y_train_grayscale, verbose=0)
print("Erro da CNN: %.2f%%" % (100-scores[1]*100))
print("Acurácia da CNN: %.2f%%" % (scores[1]*100))
X_train_grayscale = None
Y_train_grayscale = None
x_val_grayscale = None
y_val_grayscale = None

# HSV_GrayScale

In [ ]:
X_frutas, Y_frutas = get_dados_hsv_grayscale(path_treino, 100)
X_train_hsv_grayscale = np.asarray(X_frutas).astype(np.float32)
Y_train_hsv_grayscale = np.asarray(Y_frutas)
X_train_hsv_grayscale = X_train_hsv_grayscale.astype('float32')/255
num_frutas = len(list(set(Y_train_hsv_grayscale)))
print(num_frutas)
Y_train_hsv_grayscale = to_categorical(Y_train_hsv_grayscale, num_frutas)

X_train_hsv_grayscale, x_val_hsv_grayscale, Y_train_hsv_grayscale, y_val_hsv_grayscale = train_test_split(X_train_hsv_grayscale, Y_train_hsv_grayscale, test_size=0.2, random_state=42)

In [ ]:
model = CNN_AlexNet(num_frutas,100,4)
#plot_model(model, show_shapes=True, show_layer_names=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
checkpoint = ModelCheckpoint(filepath='./modelo_cnn_AlexNet_8_1.hdf5', verbose=0,  save_best_only=True, monitor='val_accuracy')
history = model.fit(X_train_hsv_grayscale, Y_train_hsv_grayscale, validation_data = (x_val_hsv_grayscale, y_val_hsv_grayscale), epochs=50, batch_size=60, callbacks=[checkpoint])
plot_history(history)
scores = model.evaluate(X_train_grayscale, Y_train_grayscale, verbose=0)
print("Erro da CNN: %.2f%%" % (100-scores[1]*100))
print("Acurácia da CNN: %.2f%%" % (scores[1]*100))
X_train_hsv_grayscale = None
Y_train_hsv_grayscale = None
x_val_hsv_grayscale = None
y_val_hsv_grayscale = None

#model = CNN_AlexNet(num_frutas,224,4)
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#history = model.fit(X_train, Y_train, validation_split = 0.2, epochs=10, batch_size=60)
#plot_history(history)
#scores = model.evaluate(X_train, Y_train, verbose=0)
#print("Erro da CNN: %.2f%%" % (100-scores[1]*100))
#print("Acurácia da CNN: %.2f%%" % (scores[1]*100))

# Teste

In [ ]:
X_frutas, Y_frutas = get_dados(path_teste, 100)
X_test = np.asarray(X_frutas).astype(np.float32)
Y_test = np.asarray(Y_frutas)
X_test = X_test.astype('float32')/255
Y_test = to_categorical(Y_test, num_frutas)
model = load_model('./modelo_cnn_8.hdf5')
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Erro da CNN: %.2f%%" % (100-scores[1]*100))
print("Acurária da CNN: %.2f%%" % (scores[1]*100))

In [ ]:
X_frutas, Y_frutas = get_dados_grayscale(path_teste, 28)
X_test_grayscale = np.asarray(X_frutas).astype(np.float32)
Y_test_grayscale = np.asarray(Y_frutas)
X_test_grayscale = X_test_grayscale.astype('float32')/255
Y_test_grayscale = to_categorical(Y_test_grayscale, num_frutas)
model = load_model('./modelo_cnn_LeNet_8.hdf5')
scores = model.evaluate(X_test_grayscale, Y_test_grayscale, verbose=0)
print("Erro da CNN: %.2f%%" % (100-scores[1]*100))
print("Acurária da CNN: %.2f%%" % (scores[1]*100))

In [ ]:
X_frutas, Y_frutas = get_dados_hsv_grayscale(path_teste, 28)
X_test_hsv_grayscale = np.asarray(X_frutas).astype(np.float32)
Y_test_hsv_grayscale = np.asarray(Y_frutas)
X_test_hsv_grayscale = X_test_hsv_grayscale.astype('float32')/255
Y_test_hsv_grayscale = to_categorical(Y_test_hsv_grayscale, num_frutas)
model = load_model('./modelo_cnn_AlexNet_8.hdf5')
scores = model.evaluate(X_test_hsv_grayscale, Y_test_hsv_grayscale, verbose=0)
print("Erro da CNN: %.2f%%" % (100-scores[1]*100))
print("Acurária da CNN: %.2f%%" % (scores[1]*100))

In [ ]:
print ("Fim Execução : ")
print (datetime.now().strftime("%Y-%m-%d %H:%M:%S"))